Hmmmm, It seems like all the information I need (e.g. chromosome number and position) is contained within the transcript type objects from plastid,but how do I extract that information. 

Perhaps for the fly data I can just take the first little part of the name when converted to a string to get the chromosome and then add on the region from attributes? Maybe an even smarter way would be to just say save everything until there is a semicolon since that is when it switches? 

For the ecoli data, this might be considerably easier because there is only one chromosome. 

Oh crud, what am I going to do about transcripts of differing lengths???

### Import the necessary packages

In [1]:
# Import necessary packages
from plastid import BAMGenomeArray,GTF2_TranscriptAssembler,Transcript, ThreePrimeMapFactory
import numpy as np
import pandas as pd
import dif_utils as dif
from plastid.plotting.plots import *
import setup_utils as st
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

### Define extra functions

In [2]:
def find_transcript(gene,transcripts):
    '''
    A function that takes the name of a gene as input and finds 
    the corresponding transcript from a transcript list. 
    
    returns both the transcript in question and the vector of counts for that transcript.
    
    This function is still a work in progress as for now it simply gives the last 
    transcript in the list that matches the gene ID. 
    '''
    
    for i in transcripts:
        if i.attr['gene_name'] == gene:
            my_transcript = i
                
    return my_transcript

### Define Paths

In [3]:
# Define the path to important files
data_path = "/home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/seleno_seq/"
gtf_path = "/home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/genome/mouse/"
target_path = '/home/keeganfl/Desktop/Work_Fall_2021/data_tables/'
save_path = '/home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/seleno_seq/'

### Set the random seed

In [4]:
np.random.seed(1938)

### Load in transcripts and remove non protein coding tanscripts

In [5]:
# load the transcript annotations from the GTF file.
# GTF2_TranscriptAssembler returns an iterator, so here we convert it to a list.
transcripts = list(GTF2_TranscriptAssembler(open(gtf_path + "mm10.refGene.gtf"),
                                                     return_type=Transcript))

In [6]:
transcripts[2].attr

{'gene_name': 'Rp1',
 'source': 'refGene',
 'transcript_id': 'NM_001195662',
 'score': '.',
 'gene_id': 'Rp1',
 'cds_genome_end': 4409187,
 'cds_genome_start': 4292980,
 'type': 'mRNA'}

In [7]:
#504 and 505 are different transcripts of the same gene with different lengths.
#If you want to test something you can do it on that. 

### Load up the target genes

In [8]:
# Load up the target gene names. 
seleno_table = pd.read_csv(target_path + 'Selenoprotein_table.csv', names = ["gene_ID"])

### Create the basis for the target portion of the bed file

In [30]:
transcripts[51].cds_start

227

In [9]:
chromo =[]
start =[]
end =[]
for gene in seleno_table["gene_ID"]:
    try:
        transcript = find_transcript(gene, transcripts)
    except:
        print(gene)
    chromo.append(str(transcript).partition(":")[0])
    start.append(transcript.attr['cds_genome_start'])
    end.append(transcript.attr['cds_genome_end'])

In [10]:
targets = pd.DataFrame(zip(chromo,start,end))
targets.columns = ["chr","start","stop"]

### Take random subsamples of all other transcripts

In [11]:
ran_tran = np.random.choice(transcripts, 2200)

---------------------------------------------------------------------------
VisibleDeprecationWarning
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of
lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated.
If you meant to do this, you must specify 'dtype=object' when creating the
ndarray.
in /tmp/ipykernel_485386/3160264374.py, line 1:

1 ran_tran = np.random.choice(transcripts, 2200)

---------------------------------------------------------------------------


In [12]:
chromo =[]
start =[]
end =[]
for transcript in ran_tran:
    if transcript.attr['transcript_id'].startswith('NR') == False:
        chromo.append(str(transcript).partition(":")[0])
        start.append(transcript.attr['cds_genome_start'])
        end.append(transcript.attr['cds_genome_end'])

In [13]:
non_targets = pd.DataFrame(zip(chromo,start,end))
non_targets.columns = ["chr","start","stop"]

### Create and save the bed file.

In [14]:
to_bed = pd.concat([targets,non_targets])

In [15]:
to_bed.to_csv(save_path + "mmus_subset.bed", sep = "\t", header = False, index = False)

### Perform the subsetting

In [16]:
! cd /home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/seleno_seq/ && \
    samtools view -b -L mmus_subset.bed Trspfl_RPF_1_Aligned.sortedByCoord.out.bam > subset_Trspfl_RPF_1.bam && \
    samtools view -b -L mmus_subset.bed control_RPF_1_Aligned.sortedByCoord.out.bam > subset_control_RPF_1.bam && \
    samtools view -b -L mmus_subset.bed Trspfl_RNA_1_Aligned.sortedByCoord.out.bam > subset_Trspfl_RNA_1.bam && \
    samtools view -b -L mmus_subset.bed control_RNA_1_Aligned.sortedByCoord.out.bam > subset_control_RNA_1.bam && \
    samtools view -b -L mmus_subset.bed Trspfl_RPF_2_Aligned.sortedByCoord.out.bam > subset_Trspfl_RPF_2.bam && \
    samtools view -b -L mmus_subset.bed control_RPF_2_Aligned.sortedByCoord.out.bam > subset_control_RPF_2.bam && \
    samtools view -b -L mmus_subset.bed Trspfl_RNA_2_Aligned.sortedByCoord.out.bam > subset_Trspfl_RNA_2.bam && \
    samtools view -b -L mmus_subset.bed control_RNA_2_Aligned.sortedByCoord.out.bam > subset_control_RNA_2.bam

### This does not look quite right, could have been caused by the changes in P-site offsetting or any of a number of things. That being said, I thing I can view this as a success

In [17]:
! cd /home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/seleno_seq/ && \
    samtools index subset_Trspfl_RPF_1.bam && \
    samtools index subset_control_RPF_1.bam && \
    samtools index subset_Trspfl_RNA_1.bam && \
    samtools index subset_control_RNA_1.bam && \
    samtools index subset_Trspfl_RPF_2.bam && \
    samtools index subset_control_RPF_2.bam && \
    samtools index subset_Trspfl_RNA_2.bam && \
    samtools index subset_control_RNA_2.bam

/bin/bash: line 2: \: command not found
